In [13]:
import pandas as pd
df= pd.read_json('dblp-ref-0.json', lines=True)
print(df)

                                                 abstract  \
0       The purpose of this study is to develop a lear...   
1       This paper describes the design and implementa...   
2       This article applied GARCH model instead AR or...   
3                                                     NaN   
4                                                     NaN   
...                                                   ...   
999995                                                NaN   
999996  In the last few years, workflow systems have b...   
999997  There are many different designs for audio amp...   
999998  This paper proposes a language acquisition fra...   
999999  Abstract   This thesis investigates the mathem...   

                                                  authors  n_citation  \
0       [Makoto Satoh, Ryo Muramatsu, Mizue Kayama, Ka...           0   
1                             [Gareth Beale, Graeme Earl]          50   
2       [Altaf Hossain, Faisal Zaman, Mohammed N

In [14]:
df.dropna(subset = ["references"], inplace=True)
df.dropna(subset = ["abstract"], inplace=True)
df=df[df['references'].map(lambda d: len(d)) > 0]
df=df.reset_index(drop=True)

In [15]:
#To limit the number of input
data=df.head(100)
print(data)

                                             abstract  \
0   The purpose of this study is to develop a lear...   
1   This paper describes the design and implementa...   
2   This article applied GARCH model instead AR or...   
3   Recent achievements in Natural Language Proces...   
4   Most applications of the abstract interpretati...   
..                                                ...   
95  In this paper we present an original semi-supe...   
96  The morphological changes of the vertebrae ass...   
97  Automated system deployment frameworks and con...   
98  We study the role of help in Non-Interactive Z...   
99  Multiple datasets that add high value to biome...   

                                              authors  n_citation  \
0   [Makoto Satoh, Ryo Muramatsu, Mizue Kayama, Ka...           0   
1                         [Gareth Beale, Graeme Earl]          50   
2   [Altaf Hossain, Faisal Zaman, Mohammed Nasser,...          50   
3   [Ankita Brahmachari, Priya Singh, A

In [1]:
#Collect important information
doc_list = list(data['id'].values)
ref_list = list(data['references'].values)
citation_list = list(data['n_citation'].values)
abstract_list = list(data['abstract'].values)
count = len(doc_list)
print(len(doc_list))

print(len(abstract_list),abstract_list)
print(len(abstract_list[2]))

NameError: name 'data' is not defined

In [17]:
from top2vec import Top2Vec
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import strip_tags

def bigrammer(doc):
    sentence_stream = simple_preprocess(strip_tags(doc), deacc=True)
    return bigram_phraser[sentence_stream]

sentence_stream = [doc.split(" ") for doc in  abstract_list]
bigram = Phrases(sentence_stream, min_count=5, threshold=100, delimiter=b' ')
bigram_phraser = Phraser(bigram)


model = Top2Vec(abstract_list,embedding_model='universal-sentence-encoder',tokenizer=bigrammer,speed="deep-learn")
model.save("Topic-model-dblp-ref-0")

print(model.get_num_topics())

100 ['The purpose of this study is to develop a learning tool for high school students studying the scientific aspects of information and communication net- works. More specifically, we focus on the basic principles of network proto- cols as the aim to develop our learning tool. Our tool gives students hands-on experience to help understand the basic principles of network protocols.', "This paper describes the design and implementation of a methodology for the visualisation and hypothetical virtual reconstruction of Roman polychrome statuary for research purposes. The methodology is intended as an attempt to move beyond visualisations which are simply believable towards a more physically accurate approach. Accurate representations of polychrome statuary have great potential utility both as a means of illustrating existing interpretations and as a means of testing and revising developing hypotheses. The goal of this methodology is to propose a pipeline which incorporates a high degree o

ImportError: universal-sentence-encoder is not available.

Try: pip install top2vec[sentence_encoders]

Alternatively try: pip install tensorflow tensorflow_hub tensorflow_text

In [ ]:
print(model.get_topics())
print(len(abstract_list))
print(abstract_list[9])
print(abstract_list[19])

In [ ]:
# Make the raw citation network by networkx
import networkx as nx
G = nx.DiGraph()

for i in range(count):
    G.add_node(doc_list[i],citation=citation_list[i],keywords=abstract_list[i])
    for j in range(len(ref_list[i])):
        G.add_node(ref_list[i][j])
        G.add_edge(ref_list[i][j],doc_list[i])
print(nx.info(G))

In [ ]:
def graph_cleaner(graph,attribute):
    r=nx.DiGraph.to_undirected(graph)
    node_list=[]
    v= nx.get_node_attributes(r,attribute)
    for node in r.nodes:
        if node not in v:
            node_list.append(node)
    graph.remove_nodes_from(node_list)

T=G.copy()
print("before graph processing:",nx.info(T))
remove = [node for node, degree in dict(T.degree()).items() if degree ==0]
T.remove_nodes_from(remove)
graph_cleaner(T,"keywords")
remove = [node for node, degree in dict(T.degree()).items() if degree ==0]
T.remove_nodes_from(remove)
print("after graph processing:",nx.info(T))


In [ ]:
#functions for calculating the impact influenced by https://link.springer.com/content/pdf/10.1007/s11192-021-04063-1.pdf

def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(set(list1)) + len(set(list2))) - intersection
    return float(intersection) / union

## p2 cited p1, and p1 influenced p2
def paper_citation_relation(g,p1,p2,f1,f2,c1,c2):
    if g.has_edge(p1,p2) and (c1+c2)!=0:
       return (c1/(c1+c2))+jaccard_similarity(f1, f2)
    else: return 0
#f1 is the list of keywords of paper p1
def keyword_citation_relation_raw(fi,fj,f1,f2):
    # if order is important
    #return 1/((f1.index(fi)+1)*(1+f2.index(fj)))
    #if not
    return 1
def keyword_citation_relation_pre(g,p1,p2,f1,f2,c1,c2,fi,fj):
    w_pp=paper_citation_relation(g,p1,p2,f1,f2,c1,c2)
    w_ca_raw=keyword_citation_relation_raw(fi,fj,f1,f2)
    sum=0
    for i in f1:
        for j in f2:
            sum=sum+keyword_citation_relation_raw(i,j,f1,f2)
    return w_pp* w_ca_raw/sum

def keyword_citation_relation(g,fi,fj):
    influence=0
    k=nx.get_node_attributes(g,"keywords")
    c=nx.get_node_attributes(g,"citation")
    if fi != fj:
        for node in g.nodes:
            if fi in k[node]:
                c1=c[node]
                f1=k[node]
                for neighbor in g.neighbors(node):
                    if fj in k[neighbor]:
                        c2=c[neighbor]
                        f2=k[neighbor]
                        influence=influence+keyword_citation_relation_pre(g,node,neighbor,f1,f2,c1,c2,fi,fj)
        return influence
    else: return 1

In [ ]:

#Test the result
print(keyword_citation_relation(T,"Machine learning","Computer vision"))
print(keyword_citation_relation(T,"Computer vision","Machine learning"))

print(keyword_citation_relation(T,"Machine learning","Reinforcement learning"))
print(keyword_citation_relation(T,"Reinforcement learning","Machine learning"))

print(keyword_citation_relation(T,"Reinforcement learning","Computer vision"))
print(keyword_citation_relation(T,"Computer vision","Reinforcement learning"))

print(keyword_citation_relation(T,"Machine learning","Machine learning"))